In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [2]:
data = pd.read_csv('Demographic_Data.csv')
data.head()

,in-store,age,items,amount,region
0,0,37,4,281.03,2
1,0,35,2,219.51,2
2,1,45,3,1525.70,4
3,1,46,3,715.25,3
4,1,33,4,1937.50,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   in-store  80000 non-null  int64  
 1   age       80000 non-null  int64  
 2   items     80000 non-null  int64  
 3   amount    80000 non-null  float64
 4   region    80000 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 3.1 MB


In [4]:
data = data.drop_duplicates()
print(data.isnull().sum())

in-store    0
age         0
items       0
amount      0
region      0
dtype: int64


In [5]:
header = data.dtypes.index
print(header)

Index(['in-store', 'age', 'items', 'amount', 'region'], dtype='object')


In [6]:
#Trouble with this discretiser, get an error based on feature_engine
#from feature_engine.discretisation import DecisionTreeDiscretiser

In [7]:
from sklearn.preprocessing import KBinsDiscretizer
from feature_engine.discretisation import EqualFrequencyDiscretiser

In [8]:
#discretizer = EqualFrequencyDiscretiser(q=5, variables = ['age'])
#data.discretizer()

In [9]:
#Didn't Work right
#discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans', variables = 'age')

In [10]:
data.head()

,in-store,age,items,amount,region
0,0,37,4,281.03,2
1,0,35,2,219.51,2
2,1,45,3,1525.70,4
3,1,46,3,715.25,3
4,1,33,4,1937.50,1


In [11]:
data.columns

Index(['in-store', 'age', 'items', 'amount', 'region'], dtype='object')

In [12]:
#group_X = ['in-store', 'items', 'amount', 'region']
#group_y = ['age']
#new_cols = group_X+group_y

In [13]:
#set(daa.columns) == set(new_cols)

In [14]:
#data_df_2 = data_df(new_cols)


In [15]:
#features data, X/independent variables
X = data.iloc[:,[0,1,2,4]]
X.head()

,in-store,age,items,region
0,0,37,4,2
1,0,35,2,2
2,1,45,3,4
3,1,46,3,3
4,1,33,4,1


In [16]:
#dependent variable/y
#y = data.iloc[:,5:5] Doesn't work, nor does 
#y = data[5:4]
y = pd.qcut(data['amount'],q=2, labels=[0,1])
y.head()

0    0
1    0
2    1
3    1
4    1
Name: amount, dtype: category
Categories (2, int64): [0 < 1]

In [17]:
y.tail()

79995    0
79996    1
79997    0
79998    0
79999    0
Name: amount, dtype: category
Categories (2, int64): [0 < 1]

In [18]:
#Splitting data for train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=123)

In [19]:
#modeling (classification)
algo = GradientBoostingClassifier("exponential", .5, 1000)
model = algo.fit(X_train, y_train)

/home/pyrot/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass loss=exponential, learning_rate=0.5, n_estimators=1000 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [20]:
#predictions
preds = model.predict(X_test)

In [21]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.83      0.68      0.75     11971
           1       0.73      0.86      0.79     12023

    accuracy                           0.77     23994
   macro avg       0.78      0.77      0.77     23994
weighted avg       0.78      0.77      0.77     23994



In [22]:
#from sklearn.tree import plot_tree
#fig = plt.figure(figsize=(25,20))
#tree = plot_tree(model, feature_names=X.columns, class_names=['0','1','2','3'], filled=True, max_depth=None)

In [23]:
#Max depth of none doesn't halt in a timely manner, nor did 13
#Built up from max depth of 5, and 11 is about as long a time as I'm willing to take.


In [24]:
#fig = plt.figure(figsize=(25,20))
#tree = plot_tree(model, feature_names=X.columns, class_names=['0','1','2','3'], filled=True, max_depth=3)

In [25]:
##Decision Tree classifier based on Age, predictive to about 29% with 4 age buckets (equal distro)
##Down to 13% with 10 buckets, 55% with 2, 38% with 3,100% with one bucket (test case)
##24% with 5, 21% with 6, 18% at 7...  trending down to 13% with 10. 2 age buckets (55%) seems best for Decision Tree
##passed entropy instead of gini, and got it up to 56%.  Passed gini/random, got about 56%
##entropy/random, 55%.  Entropy/random/none/5, got 55% with 63% on the younger cohort.
##gini/best/7/5 went to 63% (67% for younger); gini/best/10/2 63%; g/b/10/5 63%; e/r/3/10 64%
#all this is based on a two-bucket age.  It was the best when I tried it, but didn't alter the buckets with the parameters


In [26]:
##RandomForestClassifier, based on Age.  With defaults passed, about 55% accuracy
##100, entropy, 3, 10 it went to 63%.  100/e/5/10 stayed 63% 100/e/5/2 it got to 64%
##100/g/5/2 is the best I got, same as 100/e/5/2 at 64%
#all this is based on a two-bucket age.  It was the best when I tried it, but didn't alter the buckets with the parameters



In [27]:
##GradientBoostingClassifier based on Age.  Passing defaults got to 63%
##changing to exponential vs deviance and boosting the number of estimators up to 1000 didn't do anything
##Going to try more buckets== changed to 4 buckets, accuracy with defaults dropped to 37%, 3 back to 46%...
##Staying with 2 buckets, as much as I find that problematic logically.



In [28]:
##DecisionTreeClassifier, based on Amount. With defaults, 4 buckets did 47%, 3 buckets 61%, 2 buckets 76%
##e/r/3/2 got 77%, e/r/3/10 at 77%, g/b/3/2 at 77%, going to max_depth 4 got 75%.  Best sticking to max_depth 3



In [29]:
##RandomForest based on Amount.  With defaults & two buckets, 76%
##100/e/5/2 went to 77%, same as 100/g/5/2 same as 1000/g/5/2, same as 100/g/7/2, same as 100/g/7/10
#Sticking with 100/g/5/2 at 77%

In [30]:
#GBC based on Amount.  Defaults at 77%
#Messing with exponential, learning rate, n_estimators did not alter from 77% 



In [31]:
#Based on what I've seen, age is less predictive than amount, with a max accuracy of around 77% prediction for 3 depth decision trees in either single or random forest classification, and that about equivalents to the gradient boosting.


In [32]:
from sklearn.model_selection import cross_val_score

In [33]:
model = DecisionTreeClassifier()
print(cross_val_score(model, X_train, y_train, cv=3))

[0.75490301 0.75656414 0.75413965]


In [34]:
model = GradientBoostingClassifier()
print(cross_val_score(model, X_train, y_train, cv=3))

[0.77231808 0.77585468 0.77177   ]


In [35]:
model = RandomForestClassifier()
print(cross_val_score(model, X_train, y_train, cv=3))

[0.75592112 0.75629622 0.75778361]


In [36]:
algos_Class = []
algos_Class.append(('Random Forest Classifier', RandomForestClassifier()))
algos_Class.append(('Decision Tree Classifier', DecisionTreeClassifier()))
algos_Class.append(('Gradient Boosting Classifier', GradientBoostingClassifier()))

In [37]:
#this code doesn't work for me, doing it the long way in next line
results = []
names = []
algos_Class = []
algos_Class.append(('Random Forest Classifier', RandomForestClassifier()))
algos_Class.append(('Decision Tree Classifier', DecisionTreeClassifier()))
algos_Class.append(('Gradient Boosting Classifier', GradientBoostingClassifier()))
for name, model in algos_Class:
    result = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
    names.append(name)
    results.append(result)
    
#for i in range(len(names)):
#print(names[i], results[i].mean()) 
    

In [38]:
for i in range(len(names)):
    print(names[i], results[i].mean()) 

Random Forest Classifier 0.7555059300012982
Decision Tree Classifier 0.7552022673462925
Gradient Boosting Classifier 0.7733142530275714


In [39]:
model = DecisionTreeClassifier()
result = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
print('Decision Tree Classifier', result.mean())
model = RandomForestClassifier()
result = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
print('Random Forest Classifier', result.mean())
model = GradientBoostingClassifier()
result = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
print('Gradient Boosting Classifier', result.mean())

Decision Tree Classifier 0.7552022673462925
Random Forest Classifier 0.7556130728503051
Gradient Boosting Classifier 0.7733142530275714


In [40]:
#Based on the above model by model example, I want to go with GradientBoosting based on Amount (2 bins)